In [114]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
%matplotlib inline

from scipy.spatial import distance


from sklearn.cluster import DBSCAN
from collections import Counter

import sys
sys.path.append('../analysis/')
from csv_to_pandas import csv_into_dict_of_data

from plot_graph import remove_zero_columns

training_dataset = "../../csv/training_dataset"
testing_dataset = "../../csv/testing_dataset"
df_key = 'snmp'
epsilon = 9000
miniminum_points = 7

In [115]:
df_dict = csv_into_dict_of_data(training_dataset)
df_dict = remove_zero_columns(df_dict)
df = df_dict[df_key]
df = df.reindex(sorted(df.columns), axis = 1)

## Dimensionality reduction  vs me as employee
### corelations & redundant??
### remove first few days

In [145]:
df1 = df.T
df = df1[df1.index > '2018-09-19']
df = df.T

df.corr()

,2018-09-20,2018-09-21,2018-09-22,2018-09-23,2018-09-24,2018-09-25,2018-09-26,2018-09-27,2018-09-28,2018-09-29,...,2019-01-22,2019-01-23,2019-01-24,2019-01-25,2019-01-26,2019-01-27,2019-01-28,2019-01-29,2019-01-30,2019-01-31
2018-09-20,1.000000,0.886437,0.903718,0.905400,0.663371,0.904782,0.900989,0.905720,0.814794,0.903441,...,-0.647422,-0.699680,-0.716183,-0.680860,-0.121662,0.046035,-0.019023,0.402428,0.123571,0.859432
2018-09-21,0.886437,1.000000,0.976720,0.976258,0.694660,0.976092,0.973821,0.975771,0.897259,0.976497,...,-0.775419,-0.816846,-0.845708,-0.807773,-0.077324,0.023614,0.091856,0.422271,0.106740,0.970178
2018-09-22,0.903718,0.976720,1.000000,0.999066,0.712693,0.998715,0.997462,0.998934,0.920362,0.999713,...,-0.769981,-0.790735,-0.833444,-0.776279,-0.043896,0.033968,0.113338,0.420501,0.115575,0.953027
2018-09-23,0.905400,0.976258,0.999066,1.000000,0.713263,0.998171,0.996453,0.998248,0.919106,0.998963,...,-0.765816,-0.787240,-0.830878,-0.772769,-0.046588,0.040332,0.109538,0.419675,0.113028,0.948854
2018-09-24,0.663371,0.694660,0.712693,0.713263,1.000000,0.725241,0.710681,0.710194,0.645186,0.714740,...,-0.436062,-0.544170,-0.518323,-0.529880,0.004201,-0.006897,0.180451,0.836777,0.207238,0.676530
2018-09-25,0.904782,0.976092,0.998715,0.998171,0.725241,1.000000,0.996910,0.997577,0.917506,0.998646,...,-0.763345,-0.789981,-0.823821,-0.770717,-0.039303,0.029069,0.118051,0.435583,0.131083,0.952113
2018-09-26,0.900989,0.973821,0.997462,0.996453,0.710681,0.996910,1.000000,0.996775,0.916835,0.997188,...,-0.764012,-0.785981,-0.823553,-0.765211,-0.032833,0.031849,0.123669,0.419920,0.128604,0.951386
2018-09-27,0.905720,0.975771,0.998934,0.998248,0.710194,0.997577,0.996775,1.000000,0.919462,0.998849,...,-0.762240,-0.783254,-0.831091,-0.769338,-0.038789,0.030938,0.111251,0.417517,0.127550,0.951676
2018-09-28,0.814794,0.897259,0.920362,0.919106,0.645186,0.917506,0.916835,0.919462,1.000000,0.918498,...,-0.717238,-0.709219,-0.760391,-0.697522,0.093720,0.079505,0.231332,0.378971,0.114721,0.870504
2018-09-29,0.903441,0.976497,0.999713,0.998963,0.714740,0.998646,0.997188,0.998849,0.918498,1.000000,...,-0.768779,-0.792604,-0.835021,-0.777797,-0.047188,0.033294,0.110633,0.423110,0.110801,0.952901
